In [10]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, FullScreenControl, DrawControl, Marker, \
                        Circle, LayersControl, Polyline, WidgetControl, LayerGroup, AwesomeIcon, Icon, ScaleControl, GeoJSON
from ipywidgets import IntSlider, VBox, HBox, Button, Dropdown, Text, Layout, IntRangeSlider, Checkbox, Label
import math
import aerocalc3.unit_conversion as convert
from pathlib import Path
import requests
import pandas as pd
import time

data_dir = Path("../data")
data_dir.mkdir(parents=True, exist_ok=True)

import pyproj

geo = pyproj.Geod(ellps="WGS84")
constant_g = 9.80665

user_lat = 15.0
user_long = -15.0

time.sleep(10)
print(user_lat)
print(user_long)

In [11]:

m = Map(
    basemap=basemaps.OpenTopoMap,
    center=(46.5, 6.5),
    scroll_wheel_zoom=True,
    zoom=5
)

m.add_control(FullScreenControl())

ee_basemaps = {}

# Loops through all ipyleaflet basemaps

for item in basemaps.values():
#     print(item.get("name", "No name."))
#     print(item.get("url", "No url."))
#     print(item.keys())
    try:
        name = item["name"]
        basemap = "basemaps.{}".format(name)
        ee_basemaps[name] = basemap_to_tiles(eval(basemap))
    except:
        try:
            for sub_item in item:
                name = item[sub_item]["name"]
                basemap = "basemaps.{}".format(name)
                basemap = basemap.replace("Mids", "Modis")
                ee_basemaps[name] = basemap_to_tiles(eval(basemap))
        except:
            pass

# Adds a Dropdown widget
dropdown = Dropdown(
    options=list(ee_basemaps.keys()),
    value="OpenTopoMap",
    description="Basemaps",
)

# Handles Dropdown control event
def on_click(change):
    basemap_name = change["new"]
    old_basemap = m.layers[-1]
    m.substitute_layer(old_basemap, ee_basemaps[basemap_name])

dropdown.observe(on_click, "value")

# Adds control to the map
basemap_control = WidgetControl(widget=dropdown, position="topright")
m.add_control(basemap_control)

# control = LayersControl(position='topleft')
# m.add_control(control)

# maison = Marker(name='Maison', location=(43.559837, 1.314936))
# m.add_layer(maison)

draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "purple",
        "weight": 10,
        "opacity": 0.2
    }
}
draw_control.polygon = {}
draw_control.circlemarker = {}
draw_control.rectangle = {}

m.add_control(draw_control)
m.add_control(ScaleControl(position='bottomleft', imperial = False, max_width = 150))



# ((lat,lon),(lat,lon))

# def toto(**args):
# #     print(args)
#     if args['type'] == "mouseup":
#         map_bounds = m.bounds
#         print(map_bounds)

# m.on_interaction(toto)

In [12]:
ourairports_runways = "https://davidmegginson.github.io/ourairports-data/runways.csv" 

ourairports_dir = data_dir / "ourairports"
ourairports_dir.mkdir(parents=True, exist_ok=True)

# class runway():
#     def __init__(self).

def on_eosid_prop_change():
    pass

class runways():
    def __init__(self, **args):

        self.rwy_df = pd.DataFrame(columns=["surface","closed","le_ident", "he_ident", "length_ft", "airport_ident", "le_latitude_deg", "le_longitude_deg", 'he_latitude_deg', "he_longitude_deg", "heading_degT", "elevation_ft" ])
        self.rwy_draw = None
        self.dep_axis = None
        self.dep = None
        
        self.rwys_display = Polyline(color="purple", fill=False, weight = 5)
        m.add_layer(self.rwys_display)
        
        self.load_btn = Button(tooltip="Download data", icon = 'download', layout=Layout(width = "1cm"))
        self.ap_t = Text(placeholder="ICAO", layout=Layout(width = "1.5cm"))
        self.ok_btn = Button(icon="check", layout=Layout(width = "1cm"))
        self.rwy_dd = Dropdown(placeholder="RWY", layout=Layout(width = "2.5cm"))

        m.add_control(WidgetControl(widget = HBox([self.load_btn, self.ap_t, self.ok_btn, self.rwy_dd]), position = "topright"))
        
        self.load_btn.on_click(self.load_fct)
        self.ok_btn.on_click(self.ok_fct)
        self.rwy_dd.observe(self.display_one)
        
        if (ourairports_dir / "runways.csv").exists():
            self.rwy_df = pd.read_csv(ourairports_dir / "runways.csv")
            self.load_btn.button_style = "success"
        else:
            self.load_btn.button_style = "danger"
        self.init_runways()

    def init_runways(self):
        if not self.rwy_df.empty:
            self.rwy_df = self.rwy_df[self.rwy_df['surface'].isin(['CON', 'ASP'])]
            self.rwy_df = self.rwy_df[self.rwy_df['closed'] == 0]

            self.rwy_df["_rwy"] = self.rwy_df["le_ident"] + "/" + self.rwy_df["he_ident"]   
            self.rwy_df['_length'] = self.rwy_df['length_ft'] * 0.3048
        
    def load_fct(self, event = None):
        self.load_btn.button_style = ""
        self.load_btn.disabled = True
        self.rwy_df = pd.read_csv(ourairports_runways)
        self.rwy_df.to_csv(ourairports_dir / "runways.csv", index=False)
        self.load_btn.disabled = False
        self.init_runways()

    def ok_fct(self, event = None):
        print("ok_fct")
        if len(self.ap_t.value) == 4:
            self.selected_rwys = self.rwy_df[self.rwy_df["airport_ident"] == self.ap_t.value.upper()].drop_duplicates()
            
            if not self.selected_rwys.empty:
                a = self.selected_rwys["le_ident"].append(self.selected_rwys["he_ident"]).to_list()

                self.rwy_dd.options = a

                r = self.selected_rwys.reset_index(drop=True).iloc[0]
                m.center = (r["le_latitude_deg"], r["le_longitude_deg"])
            else:
                self.ap_t.value = ""

    def get_runways(self, **args):

        if args['type'] == "mouseup":
            self._get_runways()
            
    def _get_runways(self, event = None):      
#             print(m.bounds)
            min_lat = m.bounds[0][0]
            min_lon = m.bounds[0][1]
            max_lat = m.bounds[1][0]
            max_lon = m.bounds[1][1]
            
            extract = self.rwy_df[(self.rwy_df['le_latitude_deg'] >= min_lat) & (self.rwy_df['he_latitude_deg'] <= max_lat) &
                        (self.rwy_df['le_longitude_deg'] >= min_lon) & (self.rwy_df['he_longitude_deg'] <= max_lon)]
                        
            locations = list()
            def _line(r):                
                locations.append(((r["le_latitude_deg"], r["le_longitude_deg"]), (r["he_latitude_deg"], r["he_longitude_deg"])))

            extract.apply(_line, axis = 1)

            self.rwys_display.locations = locations
    
 
    def display_one(self, event):
#         print(event)
        if event["name"] == "label":

            if self.dep:
                m.remove_layer(self.dep)
#                 m.remove_layer(self.rwy_draw)
#             curr_traj.reset()
            
            r = self.rwy_df[(self.rwy_df['le_ident'] == event['new']) & (self.rwy_df['airport_ident'] == self.ap_t.value.upper())] 
            th_prefix = "le_"
            opp_prefix = "he_"
            if r.empty:
                r = self.rwy_df[(self.rwy_df['he_ident'] == event['new']) & (self.rwy_df['airport_ident'] == self.ap_t.value.upper())]
                th_prefix = "he_"
                opp_prefix = "le_"
                
            
            r = r.reset_index(drop=True).iloc[0]
#             print(r)
            ### %%%%%
            
            
            self.rwy_draw = Polyline(locations = ((r["le_latitude_deg"], r["le_longitude_deg"]), (r["he_latitude_deg"], r["he_longitude_deg"]) )  , color="red", fill=False, weight = 10)
            
            
            p_lon, p_lat, _ =  geo.fwd(r[th_prefix + "longitude_deg"], r[th_prefix + "latitude_deg"], r[th_prefix + "heading_degT"], r["_length"] * 10.0 )
            
            self.dep_axis =  Polyline(locations = ((r[th_prefix + "latitude_deg"], r[th_prefix + "longitude_deg"]), (p_lat, p_lon)), color="blue", fill=False, weight = 2)
            
            ### %%%%%
            self.dep = LayerGroup(layers=[self.rwy_draw, self.dep_axis])
            
            m.add_layer(self.dep)
        
            vert_profile.set_runway(r[th_prefix + "latitude_deg"], r[th_prefix + "longitude_deg"], r[th_prefix + "elevation_ft"], r[th_prefix + "heading_degT"] , r[opp_prefix + "latitude_deg"], r[opp_prefix + "longitude_deg"], r[opp_prefix + "elevation_ft"])
            on_eosid_prop_change()
            
rwy = runways()
# m.on_interaction(rwy.get_runways)
m.observe(rwy._get_runways, "bounds")





In [13]:
def max_bank_angle(height, max_angle = 25):
    if height <= 400:
        return min(15, max_angle)
    else:
        return min(25, max_angle)     
       
def obstacle_clearance(max_bank_angle = 15):
    if max_bank_angle <= 15:
        return 35
    else:
        return 50
        
def max_splay_width(max_turn, imc = True):
    if max_turn <= 15 and imc:
        return 600
    elif 15 < max_turn  and imc:
        return 900
    
    elif max_turn <= 15 and vmc:
        return 300
    elif 15 < max_turn and vmc:
        return 600
    
    else:
        return 900
    
def splay_width(cur_dist, max_turn = 25, imc = True):
    return min((90 + cur_dist*math.tan(math.radians(7.1))), max_splay_width(max_turn, imc))
    
def turn_radius(speed_kts, angle_deg):
    
    n_spd = convert.speed_conv(speed_kts, from_units='kt', to_units="m/s")
    n_ba = math.radians(angle_deg)
    r = math.pow(n_spd, 2) / (constant_g * math.tan(n_ba))
    
    return int(convert.len_conv(r, from_units="m", to_units='m'))

In [14]:
class vertical_profile():
    # Add net_altitude
    def __init__(self):
        self.threshold_lat = None
        self.threshold_lon = None
        self.threshold_alt_ft = None
        self.threshold_height_ft = 0
        self.threshold_height_m = 0
        
        self.rwy_heading_degT = 0
        self.rwy_length_m = None
        
        self._35ft_lat = None
        self._35ft_lon = None
        self._35ft_height_ft = 35
        self._35ft_alt_ft = None
        self._35ft_height_m = convert.len_conv(self._35ft_height_ft, from_units="ft", to_units="m")
        self._35ft_cur_dist_m  = None
        
        self._50ft_lat = None
        self._50ft_lon = None
        self._50ft_height_ft = 50
        self._50ft_height_m = convert.len_conv(self._50ft_height_ft, from_units="ft", to_units="m")
        self._50ft_cur_dist_m = None
        self._50ft_marker = Marker(draggable=False, title = "50 ft",icon = Icon(icon_url='pics/green-triangle.png', icon_size=[16, 16], icon_anchor=[8,8]))
        m.add_layer(self._50ft_marker)
        
        self._400ft_lat = None
        self._400ft_lon = None
        self._400ft_height_ft = 400
        self._400ft_height_m = convert.len_conv(self._400ft_height_ft, from_units="ft", to_units="m")
        self._400ft_cur_dist_m = None
        self._400ft_marker = Marker(draggable=False, title = "400 ft", icon = Icon(icon_url='pics/green-square.png', icon_size=[16, 16], icon_anchor=[8,8]))
        m.add_layer(self._400ft_marker)
        
        self.acc_start_cur_dist_m = None
        self.acc_height_ft = None
        self.acc_height_m = None
        self.acc_length_m = None
        
        self.acc_start_marker = Marker(draggable=False, title = "Acc Start", icon = Icon(icon_url='pics/cyan-triangle-down.png', icon_size=[16, 16], icon_anchor=[8,8]))
        m.add_layer(self.acc_start_marker)
        
        self.acc_end_cur_dist_m = None
        self.acc_end_marker = Marker(draggable=False, title = "Acc End", icon = Icon(icon_url='pics/blue-triangle-up.png', icon_size=[16, 16], icon_anchor=[8,8]))
        m.add_layer(self.acc_end_marker)
        
        self._5000ft_cur_dist_m = None
        self._5000ft_height_ft = 5000
        self._5000ft_height_m = convert.len_conv(self._5000ft_height_ft, from_units="ft", to_units="m")
        
        self.v2_ms = None
        self.v2_climb_gradient = 2.4 / 100.0
        self.v2_climb_angle = math.atan(self.v2_climb_gradient)
        self.gd_climb_gradient = 1.2 / 100.0
        
        self.gd_ms = None
        
    def set_user_data(self, _v2_kt, _acc_height_ft, _acc_length_Nm, _gd_kt):
        self.v2_ms = convert.speed_conv(_v2_kt, from_units="kt", to_units="m/s")
        self.gd_ms = convert.speed_conv(_gd_kt, from_units="kt", to_units="m/s")
        
        self.acc_height_ft = _acc_height_ft
        self.acc_length_Nm = _acc_length_Nm
 
        self.acc_height_m = convert.len_conv(self.acc_height_ft, from_units="ft", to_units="m")
        self.acc_length_m = convert.len_conv(self.acc_length_Nm, from_units="nm", to_units="m")
        
    def set_runway(self, th_lat, th_lon, th_alt_ft, rw_hdg, _35ft_lat, _35ft_lon, _35ft_alt_ft):
        self.threshold_lat = th_lat
        self.threshold_lon = th_lon
        self.threshold_alt_ft = th_alt_ft
        self.threshold_height_ft = 0
        
        self._35ft_lat = _35ft_lat
        self._35ft_lon= _35ft_lon
        self._35ft_alt_ft = _35ft_alt_ft
        self._35ft_height_ft = 35
        
        _,_, self._35ft_cur_dist_m = geo.inv(self.threshold_lon, self.threshold_lat, self._35ft_lon, self._35ft_lat)
        self.rwy_length_m = self._35ft_cur_dist_m
        self.rwy_heading_degT = rw_hdg
        
        
    def compute_speed_dependance(self):
        
        dist_50ft_m = (self._50ft_height_m - self._35ft_height_m) / self.v2_climb_gradient
        self._50ft_cur_dist_m = self._35ft_cur_dist_m + dist_50ft_m
        self._50ft_lon, self._50ft_lat, _ = geo.fwd(self.threshold_lon, self.threshold_lat, self.rwy_heading_degT, self._50ft_cur_dist_m)
        
        ### %%%
        self._50ft_marker.location = (self._50ft_lat, self._50ft_lon)
        
        
        dist_400ft_m = (self._400ft_height_m - self._35ft_height_m) / self.v2_climb_gradient
        self._400ft_cur_dist_m = self._35ft_cur_dist_m + dist_400ft_m
        self._400ft_lon, self._400ft_lat, _ = geo.fwd(self.threshold_lon, self.threshold_lat, self.rwy_heading_degT, self._400ft_cur_dist_m)
        
        ### %%%
        self._400ft_marker.location = (self._400ft_lat, self._400ft_lon)
        
        dist_acc_m = (self.acc_height_m - self._35ft_height_m) / self.v2_climb_gradient
        
        self.acc_start_cur_dist_m = self._35ft_cur_dist_m + dist_acc_m 
        p_lon, p_lat, _ = geo.fwd(self.threshold_lon, self.threshold_lat, self.rwy_heading_degT, self.acc_start_cur_dist_m)
        self.acc_start_marker.location = (p_lat, p_lon)
        
        
        self.acc_end_cur_dist_m = self.acc_start_cur_dist_m + self.acc_length_m
        p_lon, p_lat, _ = geo.fwd(self.threshold_lon, self.threshold_lat, self.rwy_heading_degT, self.acc_end_cur_dist_m)
        self.acc_end_marker.location = (p_lat, p_lon)
        
        dist_5000ft_m = (self._5000ft_height_m - self.acc_height_m) / self.gd_climb_gradient 
        self._5000ft_cur_dist_m = self.acc_end_cur_dist_m + dist_5000ft_m
        
        
        p_lon, p_lat, _ =  geo.fwd(self.threshold_lon, self.threshold_lat, self.rwy_heading_degT, self._5000ft_cur_dist_m)
            
        rwy.dep_axis.locations = ((self.threshold_lat, self.threshold_lon), (p_lat, p_lon))
        
#         self._print()
      
    def estimated_height(self, curr_dist):
        if curr_dist < self._35ft_cur_dist_m:
            return self._35ft_height_m
        
        elif self._35ft_cur_dist_m <= curr_dist and curr_dist < self.acc_start_cur_dist_m:
            return self._35ft_height_m + ((self.acc_height_m - self._35ft_height_m) / (self.acc_start_cur_dist_m - self._35ft_cur_dist_m)) * (curr_dist - self._35ft_cur_dist_m)
        
        elif self.acc_start_cur_dist_m <= curr_dist and curr_dist < self.acc_end_cur_dist_m:
            return self.acc_height_m
        
        elif self.acc_end_cur_dist_m <= curr_dist and curr_dist < self._5000ft_cur_dist_m:
            return self.acc_height_m + ((self._5000ft_height_m - self.acc_height_m) / (self._5000ft_cur_dist_m - self.acc_end_cur_dist_m)) * (curr_dist - self.acc_end_cur_dist_m)
        
        else:
            return self._5000ft_height_m
        
    def estimated_speed(self, curr_dist):
        if curr_dist < self._35ft_cur_dist_m:
            return 0
        elif self._35ft_cur_dist_m <= curr_dist < self.acc_start_cur_dist_m:
            return self.v2_ms
        elif self.acc_start_cur_dist_m <= curr_dist < self.acc_end_cur_dist_m:        
            return self.v2_ms + ((self.gd_ms - self.v2_ms)/(self.acc_end_cur_dist_m - self.acc_start_cur_dist_m)) * (curr_dist - self.acc_start_cur_dist_m)
        elif self.acc_end_cur_dist_m <= curr_dist < self._5000ft_cur_dist_m:
            return self.gd_ms
        else:
            return self.gd_ms 
        
    def get_data(self):
        x = []
        y = []
        if self.acc_height_m:
            x = [0, 
                 convert.len_conv(self._35ft_cur_dist_m, from_units="m", to_units="nm"), 
                 convert.len_conv(self.acc_start_cur_dist_m, from_units="m", to_units="nm"), 
                 convert.len_conv(self.acc_end_cur_dist_m, from_units="m", to_units="nm"), 
                 convert.len_conv(self._5000ft_cur_dist_m, from_units="m", to_units="nm")]
            y = [0, 
                 convert.len_conv(self._35ft_height_m, from_units="m", to_units="ft"), 
                 convert.len_conv(self.acc_height_m, from_units="m", to_units="ft"), 
                 convert.len_conv(self.acc_height_m, from_units="m", to_units="ft"), 
                 convert.len_conv(self._5000ft_height_m, from_units="m", to_units="ft")]
        return x, y
        
        
    def _print(self):
        print('---------------------------------------------------')
        print("35ft : dist(m) : %f, height(m) : %f,  height(ft) : %f"%(self._35ft_cur_dist_m, self._35ft_height_m, convert.len_conv(self._35ft_height_m, from_units="m", to_units='ft')))
        print("50ft : dist(m) : %f, height(m) : %f,  height(ft) : %f"%(self._50ft_cur_dist_m, self._50ft_height_m, convert.len_conv(self._50ft_height_m, from_units="m", to_units='ft')))
        print("400ft : dist(m) : %f, height(m) : %f, height(ft) : %f"%(self._400ft_cur_dist_m, self._400ft_height_m, convert.len_conv(self._400ft_height_m, from_units="m", to_units='ft')))
        print("Acceleration : dist(m) %f, height(m) : %f, height(ft) : %f"%(self.acc_start_cur_dist_m, self.acc_height_m, convert.len_conv(self.acc_height_m, from_units="m", to_units='ft')))
        print("Green dot : dist(m) : %f, height(m) : %f, height(ft) : %f"%(self.acc_end_cur_dist_m, self.acc_height_m, convert.len_conv(self.acc_height_m, from_units="m", to_units='ft')))
        print("5000ft : dist(m) : %f, height(m) : %f, height(ft) : %f"%(self._5000ft_cur_dist_m, self._5000ft_height_m, convert.len_conv(self._5000ft_height_m, from_units="m", to_units='ft')))
        
vert_profile = vertical_profile()
# vert_profile.set_runway(43.12345, 0.12345, 100, 43.54321, 0.54321, 110)
# vert_profile.set_user_data(140, 1500, 10, 200)
# vert_profile.compute_speed_dependance()

        
    
    

In [15]:
class point(Marker):
    def __init__(self, **args):
        super(point, self).__init__(**args)
        self.latitude = 0
        self.longitude = 0
        self.height_m = 0
        self.height_ft = 0
        self.speed_ms = 0
        self.cur_dist_m = 0
        self.draggable = False
        self.title = ""
#         m.add_layer(self.marker)
        self.location = self._location()
        self.map = args.get("map", None)
        
    def _location(self):
        return (self.latitude, self.longitude)
    
    def pyproj(self):
        return (self.longitude, self.latitude)
    
    def plot(self):
        self.location = self._location()
        
    def splay_turn(self):
        self.icon = Icon(icon_url='pics/darkred-smalldot.png', icon_size=[16, 16], icon_anchor=[8,8])
        return self
    
    def traj_turn(self):
        self.icon = Icon(icon_url='pics/green-dot.png', icon_size=[16, 16], icon_anchor=[8,8])
        return self
    
    def acc_start(self):
        self.icon = Icon(icon_url='pics/cyan-triangle-down.png', icon_size=[16, 16], icon_anchor=[8,8])
        return self
    
    def acc_end(self):
        self.icon = Icon(icon_url='pics/blue-triangle-up.png', icon_size=[16, 16], icon_anchor=[8,8])
        return self
    
    def mct(self):
        self.icon = Icon(icon_url='pics/red-double-triangle.png', icon_size=[16, 16], icon_anchor=[8,8])
        return self
        
class turn(LayerGroup):
    def __init__(self, leg = None, traj = None, **args):
#         LayerGroup.__init__(self,args)
        super(turn, self).__init__(**args)
        self.leg = leg
        self.traj = traj
        
        self.entry = point().traj_turn()
        self.exit = point().traj_turn()
        
        self.circle= Circle( color = "green", weight=3, fill = False)
        
        self.map = args.get("map", None)
        
#         self.center_latitude = 0
#         self.center_longitude = 0
        self.center = point()
        
        self.radius_m = 0
        
        self.delta = 0
        self.entry_az = 0
        self.exit_az = 0
        self.length_m = 0
        
        self.add_layer(self.entry)
        self.add_layer(self.exit)
        self.add_layer(self.circle)
     
    def reset(self):
    
#         print(self.circle)
#         print( self.entry_marker)
#         print( self.exit_marker)
        
        
    
        if self.circle: 
            m.remove_layer(self.circle)
            self.circle = None
        if self.entry_marker: 
            m.remove_layer(self.entry)
            self.entry_marker = None
        if self.exit_marker: 
            m.remove_layer(self.exit)
            self.exit_marker = None

        
    def draw(self, m):
        
#         self.reset()
        
#         self.circle = Circle( color = "green", weight=3, fill = False)
            
        ll = list(self.traj.turns_layer.layers)
        ll.append(self)
        
        self.traj.turns_layer.layers = ll
        
        self.update()
        
        
    def update(self):
        if self.circle:
            self.circle.location = self.center._location()
            self.circle.radius = self.radius_m 
            self.entry.plot()
            self.exit.plot()
            
#             print("Circle radius : %f"%(self.circle.radius))



class leg():
    def __init__(self, traj = None, **args):

        self.traj = traj
        
        # Herited from previous leg
        self.previous_leg = None
        self.next_leg = None
        
        self.is_first = True
        self.is_last = True
        
        self.waypoint = point()
        
        self.inbound_true_track = 0
        self.inbound_dist_m = 0
        self.straight_dist_m = 0
        self.flat_penalty_m = 0
        self.user_max_bank_angle = 0
        
        self.cumulated_inbound_dist_m = 0
        
        self.outbound_true_track = 0
        self.outbound_dist_m = 0
        
        self.turn = turn(leg = self, traj = self.traj)
        
        self.right_turn = turn(leg = self, traj = self.traj)
        self.right_turn.circle.color='purple'
        self.left_turn = turn(leg = self, traj = self.traj)
        self.left_turn.circle.color='purple'
        self.splay_width_m = 0
        
    
    def set_waypoint(self, lat, lon, alt = 0):
        self.waypoint.latitude = lat
        self.waypoint.longitude = lon

        
        
    def set_previous(self, l):

        self.previous_leg = l
        l.next_leg = self
        
        self.is_first = False
        self.previous_leg.is_last = False
        
#         geo.inv(lon1, lat1, lon2, lat2) => fwd, bwd az, dist
        
        fwd_az, bwd_az, dist_m = geo.inv(self.previous_leg.waypoint.longitude, self.previous_leg.waypoint.latitude, self.waypoint.longitude, self.waypoint.latitude)
        self.previous_leg.outbound_true_track = fwd_az
        self.inbound_dist_m = dist_m
        if self.previous_leg:
            self.cumulated_inbound_dist_m = self.previous_leg.cumulated_inbound_dist_m + self.inbound_dist_m 
        else:
            self.cumulated_inbound_dist_m = self.inbound_dist_m
        
        self.previous_leg.outbound_dist_m = dist_m
        self.inbound_true_track = bwd_az
        
        self.waypoint.height_m = vert_profile.estimated_height(self.cumulated_inbound_dist_m)
        self.waypoint.height_ft = convert.len_conv(self.waypoint.height_m, from_units="m", to_units="ft")
        self.waypoint.speed_ms = vert_profile.estimated_speed(self.cumulated_inbound_dist_m)
        
    def compute(self, user_max_bank_angle):
        self.waypoint.height_m = vert_profile.estimated_height(self.cumulated_inbound_dist_m)
        self.waypoint.height_ft = convert.len_conv(self.waypoint.height_m, from_units="m", to_units="ft")
            
        self.waypoint.speed_ms = vert_profile.estimated_speed(self.cumulated_inbound_dist_m)
#         print('-----------------------------------------------------')
        self.user_max_bank_angle = user_max_bank_angle

        if (not self.is_first) and (not self.is_last):
#             print("Cas 1")
            i = self.inbound_true_track
            o = self.outbound_true_track

            self.mid_true_track = (i + o) / 2 
    
            if abs(o-i) > 180:
                self.mid_true_track = self.mid_true_track + 180
                self.delta = 360 - abs(o-i)
            else:
                self.delta = abs(o-i)
            
            self.mid_longitude, self.mid_latitude, _ = geo.fwd(self.waypoint.longitude, self.waypoint.latitude, self.mid_true_track, 5*1852)
            
#             print("Bank angle, user : %f, max : %f"%(user_max_bank_angle, max_bank_angle(self.waypoint_estimated_height_ft, user_max_bank_angle)))
            self.turn.radius_m = turn_radius(convert.speed_conv(self.waypoint.speed_ms, from_units='m/s', to_units="kt"), max_bank_angle(self.waypoint.height_ft, user_max_bank_angle)) 
        
#             print("Spd(kt) : %f, BA(°) : %f, Radius (m) : %f"%(convert.speed_conv(self.waypoint_estimated_speed_ms, from_units='m/s', to_units="kt"), max_bank_angle(self.waypoint_estimated_height_ft, user_max_bank_angle),self.turn.radius_m))
        
        
            self.dist_center_m = abs(self.turn.radius_m/math.sin(math.radians(self.delta / 2.0)))
            self.turn.center.longitude, self.turn.center.latitude, _ = geo.fwd(self.waypoint.longitude, self.waypoint.latitude, self.mid_true_track, self.dist_center_m)
            
            self.turn_dist_m = abs(self.turn.radius_m/math.tan(math.radians(self.delta / 2.0)))
    
            self.turn.entry.longitude, self.turn.entry.latitude, _ = geo.fwd(self.waypoint.longitude, self.waypoint.latitude, self.inbound_true_track, self.turn_dist_m)
            self.turn.exit.longitude, self.turn.exit.latitude, _ = geo.fwd(self.waypoint.longitude, self.waypoint.latitude, self.outbound_true_track, self.turn_dist_m)
            
            self.turn.entry_az, _,_ = geo.inv(self.turn.center.longitude, self.turn.center.latitude, self.turn.entry.longitude, self.turn.entry.latitude) 
            self.turn.exit_az, _,_ = geo.inv(self.turn.center.longitude, self.turn.center.latitude, self.turn.exit.longitude, self.turn.exit.latitude)
            
            if abs(self.turn.exit_az-self.turn.entry_az) > 180:
                self.turn.delta = 360 - abs(self.turn.exit_az-self.turn.entry_az)
            else:
                self.turn.delta = abs(self.turn.exit_az-self.turn.entry_az)
                
            self.turn.length_m = self.turn.radius_m * math.radians(self.turn.delta)    
                
            if self.previous_leg.is_first:
                _,_, self.straight_dist_m = geo.inv(self.previous_leg.waypoint.longitude, self.previous_leg.waypoint.latitude, self.turn.entry.longitude, self.turn.entry.latitude)
#                 print("First  %f %f %f %f %f %f"%(self.previous_leg.waypoint_longitude, self.waypoint_latitude, 
#                                                  self.waypoint_longitude, self.waypoint_latitude,
#                                                  self.turn.exit_longitude, self.turn.exit_latitude))
                self.turn.entry.cur_dist_m = self.straight_dist_m
                self.turn.exit.cur_dist_m = self.turn.entry.cur_dist_m + self.turn.length_m
                
            else:
                _,_, self.straight_dist_m = geo.inv(self.previous_leg.turn.exit.longitude, self.previous_leg.turn.exit.latitude, self.turn.entry.longitude, self.turn.entry.latitude)
#                 print("Then %f %f %f %f %f %f %f %f"%(self.previous_leg.turn.exit_longitude, self.previous_leg.turn.exit_latitude, 
#                                              self.turn.entry_longitude, self.turn.entry_latitude, 
#                                              self.waypoint_longitude, self.waypoint_latitude,
#                                              self.turn.exit_longitude, self.turn.exit_latitude))
            
                self.turn.entry.cur_dist_m = self.previous_leg.turn.exit.cur_dist_m + self.straight_dist_m
                self.turn.exit.cur_dist_m = self.turn.entry.cur_dist_m + self.turn.length_m
            
        
        if self.is_first:
#             print("Cas 0")
            self.straight_dist_m = 0
#             self.is_first = True
            self.turn.entry.cur_dist_m = 0
            self.turn.exit.cur_dist_m = 0
            self.turn.entry.height_m = vert_profile._35ft_height_m
            self.turn.exit.height_m = vert_profile._35ft_height_m
            
        if self.is_last:
#             print("Cas Last")
            _,_, self.straight_dist_m = geo.inv(self.previous_leg.turn.exit.longitude, self.previous_leg.turn.exit.latitude, self.waypoint.longitude, self.waypoint.latitude)
            self.turn.entry.cur_dist_m = self.previous_leg.turn.exit.cur_dist_m + self.straight_dist_m
            self.turn.exit.cur_dist_m = self.turn.entry.cur_dist_m

        self.update()     
#         print("--------------")    
#         print(self.turn)
#         print(self.turn.circle)
#         print("==============") 
#         self.update()    
#         if self.turn and self.turn.circle: 
#             print("hello")
#             self.right_entry.longitude, self.right_entry.latitude, _ = geo.fwd(self.turn.entry_longitude, self.turn.entry_latitude, self.inbound_true_track + 90, splay_width(self.turn.entry_cur_dist_m, user_max_bank_angle))
#             self.right_entry.plot()
#             self.left_entry.longitude, self.left_entry.latitude, _ = geo.fwd(self.turn.entry_longitude, self.turn.entry_latitude, self.inbound_true_track - 90, splay_width(self.turn.entry_cur_dist_m, user_max_bank_angle))
#             self.left_entry.plot()

#             self.right_exit.longitude, self.right_exit.latitude, _ = geo.fwd(self.turn.exit_longitude, self.turn.exit_latitude, self.outbound_true_track - 90, splay_width(self.turn.exit_cur_dist_m, user_max_bank_angle))
#             self.right_exit.plot()
#             self.left_exit.longitude, self.left_exit.latitude, _ = geo.fwd(self.turn.exit_longitude, self.turn.exit_latitude, self.outbound_true_track + 90, splay_width(self.turn.exit_cur_dist_m, user_max_bank_angle))
#             self.left_exit.plot() 

#             self.right_turn.radius = self.turn.circle.radius + splay_width(self.turn.entry_cur_dist_m, user_max_bank_angle)
#             self.right_turn.location = self.turn.circle.location
#             m.add_layer(self.right_turn)

#             self.left_turn.radius = self.turn.circle.radius - splay_width(self.turn.exit_cur_dist_m, user_max_bank_angle)
#             self.left_turn.location = self.turn.circle.location
#             m.add_layer(self.left_turn)
        
        
        
  
            
#             print("{%f}"%(self.turn.radius))
#             print("{%f} {%f}"%(self.inbound_true_track, self.outbound_true_track))
#             print("{%f} {%f}"%(self.delta, self.dist_center))

    def compute_height_straight_turn(self, flat_penalty_m):
#         print("Wpt       : %f m, %f Nm, %f m, %f ft"%(self.cumulated_inbound_dist_m, convert.len_conv(self.cumulated_inbound_dist_m, from_units="m", to_units="nm"), self.waypoint.height_m, convert.len_conv(self.waypoint.height_m, from_units="m", to_units="ft")))

        if self.turn:
            self.turn.entry.height_m = vert_profile.estimated_height(self.turn.entry.cur_dist_m - flat_penalty_m)
            self.turn.exit.height_m = self.turn.entry.height_m

            self.flat_penalty_m = flat_penalty_m + self.turn.length_m
            
        return self.flat_penalty_m

#             print("   Entry  : %f m, %f Nm, %f m, %f ft"%(self.turn.entry_cur_dist_m, convert.len_conv(self.turn.entry_cur_dist_m, from_units="m", to_units="nm"), self.turn.entry_height_m, convert.len_conv(self.turn.entry_height_m, from_units="m", to_units="ft") ))
        
#             print("Turn       : %f m, %f Nm"%(self.turn.radius_m, convert.len_conv(self.turn.radius_m, from_units="m", to_units="nm")))
        
#             print("   Exit   : %f m, %f Nm, %f m, %f ft"%(self.turn.exit_cur_dist_m, convert.len_conv(self.turn.exit_cur_dist_m, from_units="m", to_units="nm"), self.turn.exit_height_m, convert.len_conv(self.turn.exit_height_m, from_units="m", to_units="ft")))
#             print("Straight : %f m, %f Nm, Turn : %f m"%(self.straight_dist_m,convert.len_conv(self.straight_dist_m, from_units="m", to_units="nm"),  self.turn.length_m))
#             print("Flat penalty (m) : %f"%(flat_penalty_m))
#             print()

    def acc_start(self, level):

        if (not self.is_last) and (self.turn.exit.cur_dist_m <= vert_profile.acc_start_cur_dist_m) and (vert_profile.acc_start_cur_dist_m <= self.next_leg.turn.entry.cur_dist_m):
            level.cur_dist_m = vert_profile.acc_start_cur_dist_m + self.flat_penalty_m
            level.height_m = vert_profile.acc_height_m
            level.longitude, level.latitude, _ = geo.fwd(self.turn.exit.longitude, self.turn.exit.latitude, self.outbound_true_track, (level.cur_dist_m - self.turn.exit.cur_dist_m))
            level.plot()

        return level 
    
    def acc_end(self, level):

        if (not self.is_last) and (self.turn.exit.cur_dist_m <= vert_profile.acc_end_cur_dist_m) and (vert_profile.acc_end_cur_dist_m <= self.next_leg.turn.entry.cur_dist_m):
            level.cur_dist_m = vert_profile.acc_end_cur_dist_m + self.flat_penalty_m
            level.height_m = vert_profile.acc_height_m
            level.longitude, level.latitude, _ = geo.fwd(self.turn.exit.longitude, self.turn.exit.latitude, self.outbound_true_track, (level.cur_dist_m - self.turn.exit.cur_dist_m))
            level.plot()

        return level 

    def _splay(self, sp, max_az = 30, right = True):
        if right:
            ops = 1
        else:
            ops = -1
        
        sp.entry.splay_turn()
        sp.exit.splay_turn()
        
        self.splay_width_m = splay_width(0.5*(self.turn.entry.cur_dist_m + self.turn.exit.cur_dist_m), max_az)
        
        if (self.previous_leg) and self.previous_leg.is_first:
            if right:
                toto = self.previous_leg.right_turn
            else:
                toto = self.previous_leg.left_turn 
                
            toto.entry.longitude = self.waypoint.longitude 
            toto.entry.latitude = self.waypoint.latitude
            toto.exit.longitude = self.waypoint.longitude 
            toto.exit.latitude = self.waypoint.latitude
            toto.entry.plot()
            toto.exit.plot()
        
        if self.is_last:
            sp.entry.longitude, sp.entry.latitude, _ = geo.fwd(self.waypoint.longitude, self.waypoint.latitude, self.inbound_true_track + ops * 90, self.splay_width_m)
            sp.exit.longitude, sp.exit.latitude, _ = geo.fwd(self.turn.exit.longitude, self.turn.exit.latitude, self.outbound_true_track - ops * 90, self.splay_width_m)
            
        else:    
            sp.entry.longitude, sp.entry.latitude, _ = geo.fwd(self.turn.entry.longitude, self.turn.entry.latitude, self.inbound_true_track + ops * 90, self.splay_width_m)
            sp.exit.longitude, sp.exit.latitude, _ = geo.fwd(self.turn.exit.longitude, self.turn.exit.latitude, self.outbound_true_track - ops * 90, self.splay_width_m)

        sp.entry.plot()
        sp.exit.plot()
       
        sp.center = self.turn.center
        
        sp.radius_m = int(self.turn.circle.radius - ops * self.splay_width_m)
        
        sp.circle.radius = sp.radius_m
        sp.circle.location = self.turn.circle.location

        return sp
      
    def compute_splay(self, max_az = 30):
        self.right_turn = self._splay(self.right_turn, max_az = max_az, right = True)
        self.left_turn = self._splay(self.left_turn, max_az = max_az, right = False)
    
    def draw(self, m):

            self.turn.draw(m)
            self.right_turn.draw(m)
            self.left_turn.draw(m)
       
    def update(self):
        
#         print("Leg - waypoint : dist(m) : %f, height(ft) : %f, spd(kts) : %f"%(self.cumulated_inbound_dist_m, self.waypoint_estimated_height_ft, convert.speed_conv(self.waypoint_estimated_speed_ms, from_units='m/s', to_units="kt")))
        
        self.turn.update()
        
        self.right_turn.update()
        self.left_turn.update()
#         print("Update %f %f %f"%(self.turn.circle.radius, self.right_turn.circle.radius, self.left_turn.circle.radius))
        
        
    def set_next(self, l):
        self.next_leg = l
        
    def _print(self, doit = True):
        if doit:
#             print('%%')
            print("%% %f %f %f "%(self.cumulated_inbound_dist_m, self.waypoint_estimated_height_m, self.waypoint_estimated_height_ft))
#             print("%f %f %f"%(self.turn.entry_az, self.turn.exit_az, self.turn.delta) )
#             if self.previous_leg:
#                 print("{%f} {%f} / {%f} {%f}"%(self.previous_leg.waypoint_longitude,self.previous_leg.waypoint_latitude, self.waypoint_longitude, self.waypoint_latitude))
#                 print("{%f} {%f} >> {%f} {%f}"%(self.previous_leg.outbound_true_track, self.previous_leg.outbound_dist, self.inbound_true_track, self.inbound_dist))
#                 print("{%f} {%f} {%f}"%(self.turn.radius, self.delta, self.dist_center))
#             else:
#                 print("None None / {%f} {%f}"%(self.waypoint_longitude, self.waypoint_latitude))
#                 print("None None >> {%f} {%f}"%(self.inbound_true_track, self.inbound_dist))
#                 print("None None None")
        else:
            pass
    def straight_mesh_for_display(self, mesh_dist_m):
        data = list()
        
        n_lon = round(self.straight_dist_m / mesh_dist_m)
        right_lonlats = geo.npts(self.previous_leg.right_turn.exit.longitude, self.previous_leg.right_turn.exit.latitude, self.right_turn.entry.longitude, self.right_turn.entry.latitude, n_lon)
        left_lonlats = geo.npts(self.previous_leg.left_turn.exit.longitude, self.previous_leg.left_turn.exit.latitude, self.left_turn.entry.longitude, self.left_turn.entry.latitude, n_lon)

        for d in list(zip(right_lonlats, left_lonlats)):    
            data.append([ [d[0][0], d[0][1]], [d[1][0], d[1][1]] ])


        data.append([   [self.previous_leg.right_turn.exit.longitude, self.previous_leg.right_turn.exit.latitude], [self.previous_leg.left_turn.exit.longitude, self.previous_leg.left_turn.exit.latitude]  ])
        data.append([ [self.right_turn.entry.longitude, self.right_turn.entry.latitude]   , [self.left_turn.entry.longitude, self.left_turn.entry.latitude]  ])

        n_lat = round((self.splay_width_m * 2) / mesh_dist_m) 
        exit_pts_lonlats = geo.npts(self.previous_leg.right_turn.exit.longitude, self.previous_leg.right_turn.exit.latitude, self.previous_leg.left_turn.exit.longitude, self.previous_leg.left_turn.exit.latitude, n_lat)
        entry_pts_lonlats = geo.npts( self.right_turn.entry.longitude, self.right_turn.entry.latitude, self.left_turn.entry.longitude, self.left_turn.entry.latitude, n_lat)
        for d in list(zip(exit_pts_lonlats, entry_pts_lonlats)): 

            data.append([ [d[0][0], d[0][1]], [d[1][0], d[1][1]] ])
        
        return data
        
    def turn_mesh_for_display(self, mesh_dist_m):
        
        data = list()
        
        n = round(self.turn.length_m / mesh_dist_m)
        
        for i in range(1,n):
            print("%f %f %f"%(self.turn.entry_az, self.turn.exit_az, self.turn.delta))

            if (0 < (self.turn.entry_az - self.turn.exit_az) and (self.turn.entry_az - self.turn.exit_az) < 180) or (self.turn.exit_az - self.turn.entry_az) > 180:
                az = self.turn.entry_az - (self.turn.delta / n) * i
            else:
                az = self.turn.entry_az + (self.turn.delta / n) * i
            
            r_t_lon, r_t_lat, _ = geo.fwd(self.turn.center.longitude, self.turn.center.latitude, az, self.right_turn.radius_m)
            l_t_lon, l_t_lat, _ = geo.fwd(self.turn.center.longitude, self.turn.center.latitude, az, self.left_turn.radius_m)
            data.append([[r_t_lon, r_t_lat], [l_t_lon, l_t_lat]])
        
        
        return data
        
        
class trajectory():
    def __init__(self, **args):
        self.legs = list()
        self.acc_start = point().acc_start()
        self.acc_end = point().acc_end()
        m.add_layer(LayerGroup(layers=(self.acc_start, self.acc_end)))
#         self.mct = point().mct()
        
        self.turns_layer = LayerGroup()
        m.add_layer(self.turns_layer)
        self.drawn_traj = Polyline(color="green", fill=False, weight = 3)
        m.add_layer(self.drawn_traj)
        
        self.right_splay_traj = Polyline(color="purple", fill=False, weight = 2)
        m.add_layer(self.right_splay_traj)
        self.left_splay_traj = Polyline(color="purple", fill=False, weight = 2)
        m.add_layer(self.left_splay_traj)
        
    def reset(self):
#         for l in self.legs:
#             l.turn.reset()
        self.legs = list()
        if self.drawn_traj:
#             m.remove_layer(self.drawn_traj)
            self.drawn_traj.positions = list()
            self.right_splay_traj.positions = list()
            self.left_splay_traj.positions = list()
#             self.drawn_traj = None
        if self.turns_layer:
        
#             m.remove_layer(self.turns_layer)
            self.turns_layer.layers = list()
#             self.turns_layer = None

    
    def new_leg(self, lat, lon, alt = 0):
        
        n_l = leg(traj = self)
        n_l.set_waypoint(lat, lon, alt)
        if self.legs:
            n_l.set_previous(self.legs[-1])
        
        self.legs.append(n_l)
        
        return n_l

    def compute(self, user_max_bank_angle):
        flat_penalties = 0
        if len(self.legs) > 0:
            first = self.legs[0]
            max_az = 0

            for l in self.legs:

                az, _, _ = geo.inv(first.waypoint.longitude, first.waypoint.latitude, l.waypoint.longitude, l.waypoint.latitude)
                max_az = max(max_az, abs(az))

                l.compute(user_max_bank_angle)
                
            self.play_width_m = splay_width(max_az)       
                
            for l in self.legs:
                flat_penalties = l.compute_height_straight_turn(flat_penalties)
            
            for l in self.legs:
                self.acc_start = l.acc_start(self.acc_start)
                self.acc_end = l.acc_end(self.acc_end)
                
                l.compute_splay(max_az)
#                 self.mct = l.mct()


    
    def draw(self, m):
#         print("draw")
        self.drawn_traj.locations  = [[pt.waypoint.latitude, pt.waypoint.longitude] for pt in self.legs] 
        self.right_splay_traj.locations  = [[(pt.previous_leg.right_turn.exit.latitude, pt.previous_leg.right_turn.exit.longitude), (pt.right_turn.entry.latitude, pt.right_turn.entry.longitude)] for pt in self.legs[1:]]
        self.left_splay_traj.locations  = [[(pt.previous_leg.left_turn.exit.latitude, pt.previous_leg.left_turn.exit.longitude), (pt.left_turn.entry.latitude, pt.left_turn.entry.longitude)] for pt in self.legs[1:]]
       
        for l in self.legs:
            l.draw(m)
            

    def update(self):
#         print('--- Update ----------------------------------------------------')

        self.right_splay_traj.locations  = [[(pt.previous_leg.right_turn.exit.latitude, pt.previous_leg.right_turn.exit.longitude), (pt.right_turn.entry.latitude, pt.right_turn.entry.longitude)] for pt in self.legs[1:]]
        self.left_splay_traj.locations  = [[(pt.previous_leg.left_turn.exit.latitude, pt.previous_leg.left_turn.exit.longitude), (pt.left_turn.entry.latitude, pt.left_turn.entry.longitude)] for pt in self.legs[1:]]
       
        
        for l in self.legs:
            l.update()
        
    def _print(self, doit = True):
        for p in self.legs:
            p._print(doit)
    
    def get_raw_data(self):
        x = []
        y = []
        for l in self.legs:
            x.append(convert.len_conv(l.cumulated_inbound_dist_m, from_units="m", to_units="nm"))
            y.append(l.waypoint.height_ft)

        return x, y
    

    def get_detailled_profile(self):
        
        level_x = list()
        level_y = list()
        
        level_x.append(convert.len_conv(self.acc_start.cur_dist_m, from_units="m", to_units="nm"))
        level_y.append(convert.len_conv(self.acc_start.height_m, from_units="m", to_units="ft"))    
        level_x.append(convert.len_conv(self.acc_end.cur_dist_m, from_units="m", to_units="nm"))
        level_y.append(convert.len_conv(self.acc_end.height_m, from_units="m", to_units="ft"))
             
        
        _entry_height = [convert.len_conv(l.turn.entry.height_m, from_units="m", to_units="ft") for l in self.legs]
        _entry_cur_dist = [convert.len_conv(l.turn.entry.cur_dist_m, from_units="m", to_units="nm") for l in self.legs]
        
        _exit_height = [convert.len_conv(l.turn.exit.height_m, from_units="m", to_units="ft") for l in self.legs]
        _exit_cur_dist = [convert.len_conv(l.turn.exit.cur_dist_m, from_units="m", to_units="nm") for l in self.legs]
        
        d1 = pd.DataFrame({"cur_dist_m":_entry_cur_dist, "height_m":_entry_height})
        d2 = pd.DataFrame({"cur_dist_m":_exit_cur_dist, "height_m":_exit_height})
        d1 = d1.append(d2, ignore_index=True)
        d1 = d1.sort_values(by=['cur_dist_m'])
#         print(d1)
        
        d2 = d1.append(pd.DataFrame({"cur_dist_m":level_x, "height_m":level_y}), ignore_index=True)
        d2 = d2.sort_values(by=['cur_dist_m'])
        
        return d1, d2, level_x, level_y 
        
        
        
    def  __print(self):
        for l in self.legs:
            print("   Entry  : %f m, %f m"%(l.turn.entry.cur_dist_m, l.turn.entry.height_m))
            print("Wpt       : %f m, %f m"%(l.cumulated_inbound_dist_m, l.waypoint.height_m))
            print("   Exit   : %f m, %f m"%(l.turn.exit.cur_dist_m, l.turn.exit.height_m))
            print("Straight : %f m, Turn : %f m"%(l.straight_dist_m, l.turn.length_m))
            print()
              
 
    def compute_mesh(self, mesh_dist_m):
        
        geojson = {"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "MultiLineString", "coordinates": []}}]}
        
        coords_geojson = geojson["features"][0]["geometry"]["coordinates"]
        
#         coords_geojson.append((1.3139, 43.559)[
        
        data = list()

        for l in self.legs[1:]:
            data.extend (l.straight_mesh_for_display(mesh_dist_m))
            
            data.extend(l.turn_mesh_for_display(mesh_dist_m))
           
        coords_geojson.extend(data)
        return geojson 
            
            
curr_traj = trajectory()
    

In [16]:
# https://github.com/bqplot/bqplot/blob/master/examples/Advanced%20Plotting/Axis%20Properties.ipynb
# https://bqplot.readthedocs.io/en/latest/_generate/bqplot.pyplot.xlim.html

import bqplot.pyplot as plt
import numpy as np

vert_fig = plt.figure(title="Vertical - x=Dist (Nm), y=Height (ft)")
vert_fig.layout.height="250px"
vert_fig.layout.width="1500px"

tp = plt.plot(np.array([]), np.array([]), colors=['blue'])
rp = plt.scatter(np.array([]), np.array([]), colors=['green'], marker="diamond")
dpp = plt.scatter(np.array([]), np.array([]), colors=['red'], marker="circle")
lp = plt.scatter(np.array([]), np.array([]), colors=['blue'], marker="circle")
dp = plt.plot(np.array([]), np.array([]), colors=['red'])

def update_figure(): 
    # Theorical profile
    vp = vert_profile.get_data()
    tp.x = np.array(vp[0])
    tp.y = np.array(vp[1])

    # Route profile
    vp = curr_traj.get_raw_data()
    rp.x = np.array(vp[0])
    rp.y = np.array(vp[1])
#     rp = plt.plot(rp_x_data, rp_y_data, colors=['green'])
#     plt.xlim(0,max(tp.x.max(), rp.x.max()))

    t_p, v_p, l_x, l_y  = curr_traj.get_detailled_profile()
    dp.x = v_p["cur_dist_m"].to_numpy()
    dp.y = v_p["height_m"].to_numpy()
    
    dpp.x = t_p["cur_dist_m"].to_numpy()
    dpp.y = t_p["height_m"].to_numpy()
    
    lp.x = np.array(l_x)
    lp.y = np.array(l_y)
    

    plt.xlim(0,60)

    
# update_figure()    

# vertical_control = WidgetControl(widget=vert_plt, position="bottomright", max_width = 500, min_width = 500, max_height = 300, min_height = 300) #, max_height = 200, max_width = 400)

vertical_control = WidgetControl(widget=vert_fig, position="bottomright", max_width = 1500, min_width = 1500, max_height = 250, min_height = 250) #, max_height = 200, max_width = 400)

m.add_control(vertical_control)


In [17]:
style = {'description_width': '100px', 'slider_width':"200px", "readout_width":"100px"}
bank_angle_label = Label(value="Pilot technics")
bank_angle_slider = IntSlider(description='Max Bank (°)', min=1, max=25, value=25,  style=style)

eosid_1_control = WidgetControl(widget=VBox([bank_angle_label, bank_angle_slider]), position='topright')
m.add_control(eosid_1_control)



to_label = Label(value="Taking-off")
v2_gd_slider = IntRangeSlider(description='V2 - GD (kts)', min=100, max=500, value=[140,200], step = 5, readout=True, readout_format='d',  style=style)
acc_height_slider = IntSlider(description='Acc height (ft)', min=400, max=5000, value=1500, step = 100, readout=True, readout_format='d',  style=style)
acc_length_slider = IntSlider(description='Acc length (Nm)', min=1, max=25, value=10, readout=True, readout_format='d',  style=style)

eosid_2_control = WidgetControl(widget=VBox([to_label, bank_angle_slider, v2_gd_slider, acc_height_slider, acc_length_slider]), position='topright')
m.add_control(eosid_2_control)

relief_label = Label(value="Relief")
mesh_length_slider = IntSlider(description='Mesh size (m)', min=100, max=1000, value=300, step = 100, readout=True, readout_format='d',  style=style)
relief_action = Button(value = False, description = "Compute relief")

eosid_3_control = WidgetControl(widget=VBox([relief_label, mesh_length_slider, relief_action]), position='topright')
m.add_control(eosid_3_control)

def handle_draw(target, action, geo_json):
#     print('handle_draw')
    curr_traj.reset()
    
    if old_geojson:
        m.remove_layer(old_geojson)
    
    if action != "deleted":
#         print('\thandle_draw::go')

        #         print("%f %f %f %f"%(v2_gd_slider.value[0], acc_height_slider.value, acc_length_slider.value, v2_gd_slider.value[1]))
        vert_profile.set_user_data(v2_gd_slider.value[0], acc_height_slider.value, acc_length_slider.value, v2_gd_slider.value[1])
        vert_profile.compute_speed_dependance()
    
        if vert_profile._35ft_lat:
            curr_traj.new_leg(vert_profile.threshold_lat, vert_profile.threshold_lon)
            curr_traj.new_leg(vert_profile._35ft_lat, vert_profile._35ft_lon)
            curr_traj.new_leg(vert_profile._50ft_lat, vert_profile._50ft_lon)
#             curr_traj.new_leg(vert_profile._400ft_lat,vert_profile._400ft_lon)


        for pt in geo_json['geometry']['coordinates']:
            curr_traj.new_leg(pt[1], pt[0])

        curr_traj.compute(bank_angle_slider.value)
        update_figure()

        
        curr_traj.draw(m)
    #     print('---------------------------------------------------------')
        curr_traj._print(False)
        compute_mesh()
        
draw_control.on_draw(handle_draw)

def on_eosid_prop_change(change = None):   
#     print("on_eosid_prop_change")
    #_v2_kt, _acc_height_ft, _acc_length_Nm, _gd_kt    
#     print(">> %f %f %f %f %f"%(bank_angle_slider.value, v2_gd_slider.value[0], acc_height_slider.value, acc_length_slider.value, v2_gd_slider.value[1]))
    vert_profile.set_user_data(v2_gd_slider.value[0], acc_height_slider.value, acc_length_slider.value, v2_gd_slider.value[1])
    vert_profile.compute_speed_dependance()
    
#     curr_traj.compute(turn_radius((v2_gd_slider.value, bank_angle_slider.value))
    curr_traj.compute(bank_angle_slider.value)
    curr_traj.update()
    update_figure()
    
#     print('===========================================================')
    curr_traj._print(False)
    compute_mesh()
    
old_geojson = None

def compute_mesh(event=False):
    global old_geojson

    geojson = curr_traj.compute_mesh(mesh_length_slider.value)
        
    new_geojson = GeoJSON(data = geojson, style={'color': 'purple', "weight":1}) #, style = {"color":"blue"} )
#     print(new_geojson)
    try:
        m.substitute_layer(old_geojson, new_geojson)
    except:
        m.add_layer(new_geojson)
    old_geojson = new_geojson


bank_angle_slider.observe(on_eosid_prop_change, 'value')
v2_gd_slider.observe(on_eosid_prop_change, 'value')
acc_height_slider.observe(on_eosid_prop_change, 'value')
acc_length_slider.observe(on_eosid_prop_change, 'value')
mesh_length_slider.observe(compute_mesh, 'value')
relief_action.on_click(compute_mesh)


rwy.ap_t.value = "LFBO"
rwy.ok_fct()

In [18]:

m

Map(center=[46.5, 6.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…